In [ ]:
from IPython.display import display, Javascript, HTML
import ipywidgets as ipw
from src import widgets, utils

In [ ]:
CONFIG_FILENAME = "config/config.json"
CONFIG = utils.read_json(CONFIG_FILENAME)
OPENBIS_OBJECT_TYPES = utils.read_json("metadata/object_types.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida(
    "https://openbis-empa-dev205.ethz.ch/"
)

In [ ]:
if OPENBIS_SESSION:
    increase_buttons_size = HTML(data="".join(CONFIG["save_home_buttons_settings"]))
    analysis_vbox = widgets.CreateAnalysisWidget(OPENBIS_SESSION)

home_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_close_buttons_hbox = ipw.HBox([save_button, home_button])

In [ ]:
def close_notebook(b):
    display(Javascript(data='window.location.replace("home.ipynb")'))


def save_analysis(b):
    project_id = analysis_vbox.select_project_widget.project_dropdown.value
    if project_id == "-1":
        return
    else:
        project_collections = utils.get_openbis_collections(
            OPENBIS_SESSION, project=project_id
        )
        analysis_collection = None
        for collection in project_collections:
            if collection.props["name"] == "Analysis":
                analysis_collection = collection

        if analysis_collection is None:
            analysis_collection = utils.create_openbis_collection(
                OPENBIS_SESSION,
                type="COLLECTION",
                project=project_id,
                code="ANALYSIS_COLLECTION",
                props={"name": "Analysis"},
            )

        simulations_ids = list(analysis_vbox.select_simulations_selector.value)
        measurements_ids = list(analysis_vbox.select_measurements_selector.value)
        software_ids = list(analysis_vbox.select_software_selector.value)
        code_ids = list(analysis_vbox.select_code_selector.value)

        analysis_parents = simulations_ids + measurements_ids

        analysis_type = OPENBIS_OBJECT_TYPES["Analysis"]

        analysis_properties = {
            "name": analysis_vbox.name_textbox.value,
            "description": analysis_vbox.description_textbox.value,
            "codes": code_ids,
            "software": software_ids,
            "comments": analysis_vbox.comments_textbox.value,
        }

        analysis_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type=analysis_type,
            parents=analysis_parents,
            collection=analysis_collection,
            props=analysis_properties,
        )

        utils.upload_datasets(
            OPENBIS_SESSION,
            analysis_object,
            analysis_vbox.support_files_uploader,
            "ATTACHMENT",
        )

        # Reset widgets
        analysis_vbox.select_project_widget.project_dropdown.value = "-1"
        analysis_vbox.select_software_selector.value = []
        analysis_vbox.select_code_selector.value = []

# Create analysis record in openBIS

In [ ]:
if OPENBIS_SESSION:
    display(analysis_vbox)
    display(increase_buttons_size)

home_button.on_click(close_notebook)
save_button.on_click(save_analysis)
display(save_close_buttons_hbox)